In [1]:
import sys
import os

repo_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd())))
sys.path.append(repo_root)

print(repo_root)

/home/jackyeung99/classes/proj_energy_volatility


In [2]:
from src.trading.IBKR_trading import TradingApp
from creds import IP
from ib_insync import *
import pandas as pd

In [10]:
TradingApp()
app = TradingApp()
app.connect(, 7497, )

SyntaxError: invalid syntax (3843311686.py, line 3)

In [12]:

HOST, PORT, CLIENT_ID = IP, 7497, 

ib = IB()
ib.connect(, PORT, clientId=CLIENT_ID)


SyntaxError: invalid syntax (1434996183.py, line 4)

In [ ]:
def get_stock():
    contract = Stock(symbol="AAPL", duration="1 M", bar_size="1 day", what="TRADES")
    ib.qualifyContracts(contract)

    print(f"\n=== {symbol} | {duration} | {bar_size} | {what} ===")
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',       # now
        durationStr=duration,
        barSizeSetting=bar_size,
        whatToShow=what,
        useRTH=True,
        formatDate=1
    )

    df = util.df(bars)
    print(df.tail(5))  # show last few rows

In [ ]:
contract = Stock("AAPL", "SMART", "USD")

NameError: name 'Stock' is not defined